In [2]:
suppressMessages(library(tidyverse))

In [ ]:
df <- suppressMessages(read_csv("/mnt/c/Users/mayal/All/School/Spring2021/ECOL346/Project/Code/1_scrape_data/fish_with_metadata.csv"))
paste("Number of fish:", nrow(df))


eco_subset <- df[ , c("ecosystem")]
eco <- df[complete.cases(eco_subset), ]
freshwater <- subset(eco, ecosystem == "Freshwater") %>% mutate(zone = "Epipelagic", classification = "Shallow Water")
freshwater <- rename(freshwater,  `depth` = `max depth`)
saltwater <- subset(eco, ecosystem == "Saltwater")
paste("Number of freshwater fish:", nrow(freshwater))
paste("Number of saltwater fish:", nrow(saltwater))

# Add supplemental depth info 
depth_supplement <- suppressMessages(read_csv("/mnt/c/Users/mayal/All/School/Spring2021/ECOL346/Project/Code/1_scrape_data/data_collection/Musilova_et_al_2019.csv")) %>%
                        select(species, `Depth (m)`) 
depth_supplement$`Depth (m)` <- as.double(depth_supplement$`Depth (m)`)
saltwater <- left_join(saltwater, depth_supplement)
saltwater <- mutate(saltwater, depth = coalesce(`max depth`, `Depth (m)`)) %>% select(- c(`max depth`, `Depth (m)`))
depth_subset <- saltwater[ , c("depth")]
depth <- saltwater[complete.cases(depth_subset), ]
paste("Number of saltwater fish with depth info:", nrow(depth))
depth <- mutate(depth, zone = ifelse(`depth` <= 200, "Epipelagic", 
                                     ifelse(`depth` <= 1000, "Mesopelagic", 
                                            ifelse(`depth` <= 4000, "Bathypelagic", 
                                                   ifelse(`depth` <= 6000, "Abyssopelagic","Hadopelagic")))))


depth <- mutate(depth, classification = ifelse(zone == "Epipelagic", "Shallow Water",
                                              ifelse(zone == "Bathypelagic" | zone == "Abyssopelagic", "Deep Water", "Mid-Depth")))

saltwater <- subset(depth, zone == "Bathypelagic" | zone == "Abyssopelagic" | zone == "Epipelagic")
for (uniq_zone in unique(depth$zone)){
    print(paste("Number of fish in the ", uniq_zone, " zone:", nrow(subset(depth, zone == uniq_zone))))
}
freshwater <- 
shallow <- subset(depth, zone == "Epipelagic")
deep <- subset(depth, zone == "Bathypelagic" | zone == "Abyssopelagic")
all_fish <- rbind(saltwater, freshwater)

In [ ]:
write_csv(all_fish, "/mnt/c/Users/mayal/All/School/Spring2021/ECOL346/Project/Code/1_scrape_data/fish_organized/all_fish_data.csv")
write_csv(saltwater, "/mnt/c/Users/mayal/All/School/Spring2021/ECOL346/Project/Code/1_scrape_data/fish_organized/saltwater_pot.csv")
write_csv(freshwater, "/mnt/c/Users/mayal/All/School/Spring2021/ECOL346/Project/Code/1_scrape_data/fish_organized/freshwater_pot.csv")
write_csv(shallow, "/mnt/c/Users/mayal/All/School/Spring2021/ECOL346/Project/Code/1_scrape_data/fish_organized/shallow_pot.csv")
write_csv(deep, "/mnt/c/Users/mayal/All/School/Spring2021/ECOL346/Project/Code/1_scrape_data/fish_organized/deep_pot.csv")